In [1]:
from duckduckgo_search import DDGS
import os
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import sys

In [2]:
sys.path.append(os.path.abspath(".."))

In [4]:
print(f"Текущая рабочая директория: {os.getcwd()}")

Текущая рабочая директория: e:\2. Учеба\2. Магистратура\3 семестр\Распознавание образов\MLOps\MLOps\MLOps\notebooks


In [5]:
from src.data.dataset import generate_links, download_images
from src.config import links_folder, dataset_folder, animals_list

# Генерация ссылок
generate_links(animals_list, max_results=50)

# Скачивание изображений
download_images(animals_list, max_images=35)

Генерация ссылок для lion, максимум: 50
Ссылки для lion сохранены в ./data/animal_links\lion.txt
Генерация ссылок для zebra, максимум: 50
Ссылки для zebra сохранены в ./data/animal_links\zebra.txt
Генерация ссылок для rabbit, максимум: 50
Ссылки для rabbit сохранены в ./data/animal_links\rabbit.txt
Генерация ссылок для bear, максимум: 50
Ссылки для bear сохранены в ./data/animal_links\bear.txt
Генерация ссылок для cow, максимум: 50
Ссылки для cow сохранены в ./data/animal_links\cow.txt
Генерация ссылок для wolf, максимум: 50
Ссылки для wolf сохранены в ./data/animal_links\wolf.txt
Генерация ссылок для cat, максимум: 50
Ссылки для cat сохранены в ./data/animal_links\cat.txt
Генерация ссылок для dog, максимум: 50
Ссылки для dog сохранены в ./data/animal_links\dog.txt
Генерация ссылок для horse, максимум: 50
Ссылки для horse сохранены в ./data/animal_links\horse.txt
Генерация ссылок для deer, максимум: 50
Ссылки для deer сохранены в ./data/animal_links\deer.txt
Скачано: ./data/animal_data

In [30]:
print(f"Текущая рабочая директория: {os.getcwd()}")

Текущая рабочая директория: e:\2. Учеба\2. Магистратура\3 семестр\Распознавание образов\MLOps\MLOps\MLOps\notebooks


In [3]:
from src.models.train_model import train_model

train_model()

e:\2. Учеба\2. Магистратура\3 семестр\Распознавание образов\MLOps\MLOps\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/10, Loss: 1.8730109930038452
Epoch 2/10, Loss: 0.10666478716302663
Epoch 3/10, Loss: 0.0446364413946867
Epoch 4/10, Loss: 0.04770191462012008
Epoch 5/10, Loss: 0.07111444132169709
Epoch 6/10, Loss: 0.0066288251837249845
Epoch 7/10, Loss: 0.034984872210770845
Epoch 8/10, Loss: 0.013246243779576616
Epoch 9/10, Loss: 0.019078605317190522
Epoch 10/10, Loss: 0.043072414671769366
Точность модели на тестовом наборе: 75.40983606557377%
Модель mobilenetv2_100 сохранена.
Модель успешно экспортирована в ONNX: ./models/mobilenetv2_100.onnx
